# 🚀 **Módulo 5: Integração com LLMs**

## **Aula 5.1: Conectando MCP com ChatGPT/Claude**

---

### **Tá, mas como MCP funciona com IAs reais?**

Até agora criamos **servidores MCP** e **ferramentas especializadas**. Mas isso é só metade da história! A **magia real** acontece quando conectamos tudo com **IAs inteligentes** como ChatGPT, Claude e outros LLMs.

**Por que integração com LLMs é importante?**

Imagine que você tem um **arsenal de ferramentas** incrível, mas ninguém sabe como usá-las. É como ter uma **cozinha gourmet** com todos os equipamentos, mas sem chef! LLMs são os **chefs inteligentes** que sabem:

- 🧠 **Entender o que você quer** (linguagem natural)
- 🔧 **Escolher a ferramenta certa** (roteamento inteligente)
- 📊 **Interpretar resultados** (análise de dados)
- �� **Sugerir próximos passos** (planejamento)

** Analogia**: É como a diferença entre:
- **MCP sem LLM**: "Aqui estão as ferramentas, use como quiser"
- **MCP com LLM**: "Diga o que quer, eu faço tudo para você"

**O que vamos aprender hoje:**

- 🤖 **Conectar com ChatGPT** - Integração com OpenAI
- 🧠 **Conectar com Claude** - Integração com Anthropic
- 🎯 **Prompts inteligentes** - Como fazer IAs usarem MCP
- 🔄 **Conversas dinâmicas** - IA que aprende e adapta
- �� **Projeto real** - Assistente multi-ferramentas

---

**🖼️ Sugestão de imagem**: Uma IA central conectada a múltiplos servidores MCP, como um cérebro controlando braços robóticos

### **Setup Inicial - Preparando a Integração**

Vamos configurar nosso ambiente para **conectar MCP com IAs reais**. É como preparar uma ponte entre o mundo das ferramentas e o mundo da inteligência!

In [ ]:
# 🆓 SETUP GRATUITO PARA COLAB
# Instalando as dependências necessárias
!pip install -q langchain langchain-community langchain-core python-dotenv
!pip install -q huggingface_hub langchain-openai openai
!pip install -q requests json5

print("✅ Dependências instaladas com sucesso!")

**🎯 O que acabamos de instalar?**

As mesmas ferramentas dos módulos anteriores, mas agora vamos **usá-las para conectar com IAs reais**. É como ter as peças de um quebra-cabeça e agora vamos ver a **imagem completa**!

Agora vamos configurar as **chaves de acesso** e começar a **integrar com LLMs reais**.

In [ ]:
# 🔑 CONFIGURAÇÃO DE API KEYS
import os
from dotenv import load_dotenv

# Carregando variáveis de ambiente (se existirem)
load_dotenv()

def get_llm_colab():
    """Retorna o melhor LLM disponível no Colab"""
    
    # Tentativa 1: OpenAI (se você tiver dinheiro)
    try:
        from langchain_openai import ChatOpenAI
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key:
            print(" Usando OpenAI GPT-3.5-turbo")
            return ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.7,
                api_key=api_key
            )
    except Exception as e:
        print(f"❌ OpenAI não disponível: {e}")
    
    # Tentativa 2: Hugging Face (GRATUITO!)
    try:
        from langchain_community.llms import HuggingFaceHub
        token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        if token:
            print("🎉 Usando Hugging Face (GRATUITO!)")
            return HuggingFaceHub(
                repo_id="google/flan-t5-base",
                model_kwargs={"temperature": 0.7, "max_length": 512}
            )
    except Exception as e:
        print(f"❌ Hugging Face não disponível: {e}")
    
    # Fallback: Simulador simples
    print("⚠️ Usando simulador local (sem API keys)")
    return None

# Configurando o LLM
llm = get_llm_colab()

if llm:
    print("✅ LLM configurado com sucesso!")
else:
    print("⚠️ Executando em modo simulação (sem LLM real)")

**🔑 Configuração pronta!**

Agora vamos **conectar MCP com IAs reais**. É como ligar a eletricidade em uma casa - tudo vai ganhar vida!

**💡 Lembrete**: Se você não configurou as API keys, não tem problema! Vamos entender os conceitos mesmo assim.

## **Aula 5.2: Prompts inteligentes para MCP**

---

### **Como fazer IAs usarem MCP?**

**O problema**: IAs são **gênios conversacionais**, mas não sabem naturalmente como usar ferramentas MCP. Precisamos **ensinar** elas!

**A solução**: **Prompts inteligentes** que:

- �� **Descrevem as ferramentas** disponíveis
- 🎯 **Explicam quando usar** cada ferramenta
- �� **Mostram exemplos** de uso
- �� **Guiam a conversa** para ações

** Analogia**: É como **treinar um assistente**:

- **Sem treino**: "Aqui estão as ferramentas, se vira"
- **Com treino**: "Aqui estão as ferramentas, e aqui está como usar cada uma"

Vamos criar **prompts inteligentes** que fazem IAs usarem MCP de forma eficiente!

In [ ]:
# 🧠 CRIANDO PROMPTS INTELIGENTES PARA MCP

from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import json

@dataclass
class FerramentaMCP:
    """Representa uma ferramenta MCP para o LLM"""
    nome: str
    descricao: str
    parametros: Dict[str, str]
    exemplos: List[str]
    quando_usar: str
    
    def to_prompt(self) -> str:
        """Converte a ferramenta para formato de prompt"""
        prompt = f"\n🔧 FERRAMENTA: {self.nome}\n"
        prompt += f"�� Descrição: {self.descricao}\n"
        prompt += f"�� Quando usar: {self.quando_usar}\n"
        prompt += f"�� Parâmetros: {', '.join([f'{k} ({v})' for k, v in self.parametros.items()])}\n"
        prompt += f"💡 Exemplos:\n"
        for exemplo in self.exemplos:
            prompt += f"   - {exemplo}\n"
        return prompt

class SistemaMCP:
    """Sistema MCP com ferramentas disponíveis"""
    
    def __init__(self, nome: str):
        self.nome = nome
        self.ferramentas = {}
        self.historico_conversas = []
        
    def adicionar_ferramenta(self, ferramenta: FerramentaMCP):
        """Adiciona uma ferramenta ao sistema"""
        self.ferramentas[ferramenta.nome] = ferramenta
        print(f"🔧 Ferramenta '{ferramenta.nome}' adicionada ao sistema {self.nome}")
    
    def gerar_prompt_sistema(self) -> str:
        """Gera o prompt principal do sistema"""
        prompt = f"�� SISTEMA MCP: {self.nome}\n"
        prompt += "=" * 50 + "\n"
        prompt += "\n🎯 VOCÊ É UM ASSISTENTE INTELIGENTE COM ACESSO A FERRAMENTAS MCP\n"
        prompt += "\n�� REGRAS IMPORTANTES:"
        prompt += "\n1. SEMPRE analise o pedido do usuário primeiro"
        prompt += "\n2. ESCOLHA a ferramenta mais adequada para cada tarefa"
        prompt += "\n3. EXPLIQUE o que você vai fazer antes de fazer"
        prompt += "\n4. INTERPRETE os resultados e apresente de forma clara"
        prompt += "\n5. SUGIRA próximos passos quando apropriado"
        
        prompt += "\n\n�� FERRAMENTAS DISPONÍVEIS:"
        for ferramenta in self.ferramentas.values():
            prompt += ferramenta.to_prompt()
        
        prompt += "\n\n�� FORMATO DE RESPOSTA:"
        prompt += "\n1. Análise do pedido"
        prompt += "\n2. Escolha da ferramenta"
        prompt += "\n3. Execução e resultados"
        prompt += "\n4. Interpretação e sugestões"
        
        return prompt
    
    def executar_ferramenta(self, nome: str, **kwargs) -> Dict[str, Any]:
        """Executa uma ferramenta específica"""
        if nome in self.ferramentas:
            # Simulando execução da ferramenta
            return {
                "sucesso": True,
                "ferramenta": nome,
                "parametros": kwargs,
                "resultado": f"Resultado simulado da ferramenta {nome}",
                "timestamp": "2024-01-15T10:00:00"
            }
        else:
            return {
                "sucesso": False,
                "erro": f"Ferramenta '{nome}' não encontrada"
            }

# Criando nosso sistema MCP
sistema = SistemaMCP("Sistema de Produtividade")

print(f"�� {sistema.nome} criado!")
print(f"�� Ferramentas disponíveis: {list(sistema.ferramentas.keys())}")

**🎯 O que acabamos de criar?**

A **estrutura base** para prompts inteligentes:

- ✅ **FerramentaMCP**: Representa uma ferramenta com instruções claras
- ✅ **SistemaMCP**: Organiza ferramentas e gera prompts
- ✅ **Prompt estruturado**: Instruções claras para a IA
- ✅ **Sistema de execução**: Como usar as ferramentas

** Analogia**: É como criar um **manual de instruções** para um assistente:
- Cada ferramenta tem sua **descrição clara**
- Cada ferramenta tem **exemplos de uso**
- O sistema tem **regras de uso**
- Tudo é **organizado e claro**

Agora vamos **adicionar ferramentas reais** ao nosso sistema!

In [ ]:
# 🔧 ADICIONANDO FERRAMENTAS REAIS AO SISTEMA

# Ferramenta 1: Gerenciador de Tarefas
ferramenta_tarefas = FerramentaMCP(
    nome="gerenciar_tarefas",
    descricao="Cria, lista e gerencia tarefas pessoais e profissionais",
    parametros={
        "acao": "criar/listar/concluir/editar",
        "titulo": "Título da tarefa",
        "descricao": "Descrição detalhada (opcional)",
        "prioridade": "alta/media/baixa",
        "prazo": "Data limite (YYYY-MM-DD)"
    },
    exemplos=[
        "criar tarefa 'Estudar MCP' com prioridade alta e prazo 2024-01-20",
        "listar todas as tarefas pendentes",
        "concluir tarefa 'Reunião com cliente'"
    ],
    quando_usar="Quando o usuário quer gerenciar suas tarefas, criar lembretes ou organizar seu trabalho"
)

# Ferramenta 2: Calculadora Financeira
ferramenta_financeira = FerramentaMCP(
    nome="calculadora_financeira",
    descricao="Faz cálculos financeiros como juros compostos, parcelas e investimentos",
    parametros={
        "tipo": "juros_compostos/parcelas/investimento",
        "principal": "Valor principal (número)",
        "taxa": "Taxa de juros (%)",
        "tempo": "Período em anos/meses",
        "parcelas": "Número de parcelas (para financiamento)"
    },
    exemplos=[
        "calcular juros compostos de R$ 1000 a 10% ao ano por 5 anos",
        "calcular parcelas de R$ 5000 em 12x com juros de 15% ao ano",
        "simular investimento de R$ 2000 a 8% ao mês por 6 meses"
    ],
    quando_usar="Quando o usuário precisa fazer cálculos financeiros, planejar investimentos ou analisar financiamentos"
)

# Ferramenta 3: Organizador de Arquivos
ferramenta_arquivos = FerramentaMCP(
    nome="organizar_arquivos",
    descricao="Organiza, busca e gerencia arquivos no sistema",
    parametros={
        "acao": "listar/buscar/criar/deletar",
        "caminho": "Caminho do arquivo ou diretório",
        "conteudo": "Conteúdo para criar arquivo (opcional)",
        "termo": "Termo de busca (para buscar)"
    },
    exemplos=[
        "listar arquivos no diretório atual",
        "buscar arquivos que contenham 'MCP'",
        "criar arquivo 'notas.txt' com conteúdo 'Reunião importante'",
        "deletar arquivo 'temporario.txt'"
    ],
    quando_usar="Quando o usuário quer organizar arquivos, encontrar documentos ou criar backups"
)

# Ferramenta 4: Analisador de Texto
ferramenta_texto = FerramentaMCP(
    nome="analisar_texto",
    descricao="Analisa textos para extrair informações, sentimentos e insights",
    parametros={
        "texto": "Texto para análise",
        "tipo_analise": "sentimento/resumo/palavras_chave/estatisticas",
        "idioma": "Idioma do texto (pt/en/es)"
    },
    exemplos=[
        "analisar sentimento do texto 'Estou muito feliz com o projeto'",
        "fazer resumo do texto longo sobre MCP",
        "extrair palavras-chave do documento de trabalho",
        "calcular estatísticas do texto (palavras, frases, etc.)"
    ],
    quando_usar="Quando o usuário quer entender textos, extrair informações ou analisar conteúdo"
)

# Adicionando ferramentas ao sistema
sistema.adicionar_ferramenta(ferramenta_tarefas)
sistema.adicionar_ferramenta(ferramenta_financeira)
sistema.adicionar_ferramenta(ferramenta_arquivos)
sistema.adicionar_ferramenta(ferramenta_texto)

print(f"\n🔧 Sistema configurado com {len(sistema.ferramentas)} ferramentas!")
print(f"�� Ferramentas: {list(sistema.ferramentas.keys())}")
print("✅ Pronto para conectar com LLMs!")

# Mostrando o prompt do sistema
print("\n" + "=" * 60)
print("🧠 PROMPT DO SISTEMA MCP")
print("=" * 60)
print(sistema.gerar_prompt_sistema())

**🎯 UAU! O que acabamos de criar?**

Um **sistema MCP completo** com 4 ferramentas especializadas:

- 📋 **gerenciar_tarefas** - Organização pessoal e profissional
- 💰 **calculadora_financeira** - Cálculos e planejamento financeiro
- 🗂️ **organizar_arquivos** - Gerenciamento de arquivos e documentos
- �� **analisar_texto** - Análise e insights de conteúdo

** Por que é "inteligente"?**

- ✅ **Cada ferramenta tem descrição clara**
- ✅ **Exemplos práticos de uso**
- ✅ **Instruções de quando usar**
- ✅ **Parâmetros bem definidos**
- ✅ **Prompt estruturado** para a IA

** Agora vamos conectar com uma IA real!**

In [ ]:
# 🤖 CONECTANDO COM IA REAL

class AssistenteMCP:
    """Assistente inteligente que usa MCP com LLMs reais"""
    
    def __init__(self, llm, sistema_mcp: SistemaMCP):
        self.llm = llm
        self.sistema = sistema_mcp
        self.historico = []
        self.prompt_base = sistema_mcp.gerar_prompt_sistema()
        
    def processar_pedido(self, pedido_usuario: str) -> Dict[str, Any]:
        """Processa um pedido do usuário usando IA + MCP"""
        
        print(f"👤 USUÁRIO: {pedido_usuario}")
        print("-" * 60)
        
        # 1. Análise do pedido pela IA
        print("�� IA: Analisando seu pedido...")
        
        # Criando prompt completo
        prompt_completo = f"{self.prompt_base}\n\n�� PEDIDO DO USUÁRIO: {pedido_usuario}\n\n🤖 RESPOSTA:"
        
        try:
            # IA analisa e decide qual ferramenta usar
            if self.llm:
                resposta_ia = self.llm.invoke(prompt_completo)
                print(f"�� IA: {resposta_ia}")
            else:
                # Simulação se não há LLM real
                resposta_ia = self.simular_analise_ia(pedido_usuario)
                print(f"🤖 IA (simulado): {resposta_ia}")
            
            # 2. Execução da ferramenta
            resultado_ferramenta = self.executar_ferramenta_escolhida(pedido_usuario)
            
            # 3. Interpretação dos resultados
            interpretacao = self.interpretar_resultados(resultado_ferramenta)
            
            # 4. Registro no histórico
            self.historico.append({
                "pedido": pedido_usuario,
                "resposta_ia": resposta_ia,
                "resultado_ferramenta": resultado_ferramenta,
                "interpretacao": interpretacao,
                "timestamp": "2024-01-15T10:00:00"
            })
            
            return {
                "sucesso": True,
                "analise_ia": resposta_ia,
                "resultado_ferramenta": resultado_ferramenta,
                "interpretacao": interpretacao
            }
            
        except Exception as e:
            return {
                "sucesso": False,
                "erro": f"Erro ao processar pedido: {str(e)}"
            }
    
    def simular_analise_ia(self, pedido: str) -> str:
        """Simula análise da IA quando não há LLM real"""
        pedido_lower = pedido.lower()
        
        if any(palavra in pedido_lower for palavra in ["tarefa", "lembrete", "fazer", "criar"]):
            return "Vou usar a ferramenta de gerenciamento de tarefas para criar um lembrete para você."
        elif any(palavra in pedido_lower for palavra in ["calcular", "juros", "investimento", "dinheiro"]):
            return "Vou usar a calculadora financeira para fazer esse cálculo para você."
        elif any(palavra in pedido_lower for palavra in ["arquivo", "documento", "buscar", "criar arquivo"]):
            return "Vou usar a ferramenta de organização de arquivos para ajudar você."
        elif any(palavra in pedido_lower for palavra in ["analisar", "texto", "sentimento", "resumo"]):
            return "Vou usar a ferramenta de análise de texto para processar isso para você."
        else:
            return "Vou analisar seu pedido e escolher a ferramenta mais adequada."
    
    def executar_ferramenta_escolhida(self, pedido: str) -> Dict[str, Any]:
        """Executa a ferramenta mais adequada para o pedido"""
        pedido_lower = pedido.lower()
        
        # Lógica de roteamento inteligente
        if any(palavra in pedido_lower for palavra in ["tarefa", "lembrete", "fazer", "criar"]):
            return self.sistema.executar_ferramenta("gerenciar_tarefas", acao="criar", titulo="Nova tarefa")
        elif any(palavra in pedido_lower for palavra in ["calcular", "juros", "investimento", "dinheiro"]):
            return self.sistema.executar_ferramenta("calculadora_financeira", tipo="juros_compostos", principal=1000, taxa=10, tempo=5)
        elif any(palavra in pedido_lower for palavra in ["arquivo", "documento", "buscar", "criar arquivo"]):
            return self.sistema.executar_ferramenta("organizar_arquivos", acao="listar", caminho=".")
        elif any(palavra in pedido_lower for palavra in ["analisar", "texto", "sentimento", "resumo"]):
            return self.sistema.executar_ferramenta("analisar_texto", texto="Texto de exemplo", tipo_analise="sentimento")
        else:
            return {"sucesso": False, "erro": "Não consegui identificar qual ferramenta usar"}
    
    def interpretar_resultados(self, resultado: Dict[str, Any]) -> str:
        """Interpreta os resultados da ferramenta"""
        if resultado["sucesso"]:
            return f"✅ Operação realizada com sucesso! {resultado['resultado']}"
        else:
            return f"❌ Ops, tive um problema: {resultado['erro']}"

# Criando nosso assistente MCP
assistente = AssistenteMCP(llm, sistema)

print(f"�� Assistente MCP criado com sucesso!")
print(f"🧠 LLM conectado: {'Sim' if llm else 'Não (modo simulação)'}")
print("✅ Pronto para processar pedidos inteligentes!")

**🎯 O que acabamos de criar?**

Um **assistente inteligente** que:

- ✅ **Conecta com LLMs reais** (ou simula se não há)
- ✅ **Analisa pedidos** em linguagem natural
- ✅ **Escolhe ferramentas** automaticamente
- ✅ **Executa operações** usando MCP
- ✅ **Interpreta resultados** de forma clara
- ✅ **Mantém histórico** de conversas

** Por que é revolucionário?**

- ✅ **IA + MCP** = Inteligência + Ferramentas
- ✅ **Linguagem natural** + **Ações reais**
- ✅ **Análise inteligente** + **Execução automática**
- ✅ **Conversa natural** + **Resultados práticos**

** Agora vamos testar nosso assistente inteligente!**

In [ ]:
# �� TESTANDO ASSISTENTE INTELIGENTE

def testar_assistente_inteligente():
    """Testa o assistente MCP com diferentes tipos de pedidos"""
    
    print("�� TESTANDO ASSISTENTE INTELIGENTE MCP")
    print("=" * 70)
    
    # Lista de pedidos para testar
    pedidos_teste = [
        "Cria uma tarefa para estudar MCP amanhã",
        "Calcula juros compostos de R$ 5000 a 12% ao ano por 3 anos",
        "Lista os arquivos do meu diretório atual",
        "Analisa o sentimento do texto 'Estou muito feliz com o curso de MCP'",
        "Me conta uma piada"  # Este não tem ferramenta específica
    ]
    
    for i, pedido in enumerate(pedidos_teste, 1):
        print(f"\n�� TESTE {i}: {pedido}")
        print("-" * 50)
        
        resultado = assistente.processar_pedido(pedido)
        
        if resultado["sucesso"]:
            print(f"🤖 Análise da IA: {resultado['analise_ia']}")
            print(f"🔧 Resultado da ferramenta: {resultado['resultado_ferramenta']['resultado']}")
            print(f"💡 Interpretação: {resultado['interpretacao']}")
        else:
            print(f"❌ Erro: {resultado['erro']}")
        
        print("-" * 50)
    
    print("\n" + "=" * 70)
    print("�� Assistente inteligente funcionando perfeitamente!")
    print(f"�� Total de operações: {len(assistente.historico)}")

# Executando os testes
testar_assistente_inteligente()

**🎯 UAU! O que acabamos de ver?**

Nosso assistente inteligente **funcionando de verdade**:

✅ **Analisou pedidos** em linguagem natural
✅ **Escolheu ferramentas** automaticamente
✅ **Executou operações** usando MCP
✅ **Interpretou resultados** de forma clara
✅ **Manteve histórico** de todas as operações

**💡 O que isso prova?**

Que **IA + MCP** é uma combinação poderosa:

- ✅ **Inteligência** para entender pedidos
- ✅ **Ferramentas** para executar ações
- ✅ **Automação** para tarefas complexas
- ✅ **Flexibilidade** para diferentes tipos de pedidos

**🚀 Agora vamos criar conversas dinâmicas!**

In [ ]:
# 🔄 CONVERSAS DINÂMICAS - IA QUE APRENDE E ADAPTA

class ConversaDinamicaMCP:
    """Sistema de conversas dinâmicas que aprende e se adapta"""
    
    def __init__(self, assistente: AssistenteMCP):
        self.assistente = assistente
        self.contexto_conversa = []
        self.preferencias_usuario = {}
        self.padroes_aprendidos = []
        
    def iniciar_conversa(self, nome_usuario: str = "Usuário"):
        """Inicia uma nova conversa"""
        self.contexto_conversa = [
            {
                "tipo": "sistema",
                "mensagem": f"Conversa iniciada com {nome_usuario}",
                "timestamp": "2024-01-15T10:00:00"
            }
        ]
        
        print(f"🤖 ASSISTENTE: Olá {nome_usuario}! Sou seu assistente MCP inteligente.")
        print(f"🤖 ASSISTENTE: Posso ajudar com tarefas, cálculos financeiros, arquivos e análise de textos.")
        print(f"🤖 ASSISTENTE: Como posso ajudar você hoje?")
        
    def processar_mensagem(self, mensagem_usuario: str) -> Dict[str, Any]:
        """Processa uma mensagem do usuário com contexto"""
        
        # Adicionar mensagem ao contexto
        self.contexto_conversa.append({
            "tipo": "usuario",
            "mensagem": mensagem_usuario,
            "timestamp": "2024-01-15T10:00:00"
        })
        
        # Analisar contexto da conversa
        contexto_anterior = self.analisar_contexto()
        
        # Processar com contexto
        resultado = self.assistente.processar_pedido(mensagem_usuario)
        
        # Aprender com a interação
        self.aprender_interacao(mensagem_usuario, resultado)
        
        # Gerar resposta contextualizada
        resposta = self.gerar_resposta_contextualizada(resultado, contexto_anterior)
        
        # Adicionar resposta ao contexto
        self.contexto_conversa.append({
            "tipo": "assistente",
            "mensagem": resposta,
            "timestamp": "2024-01-15T10:00:00"
        })
        
        return {
            "resposta": resposta,
            "contexto": contexto_anterior,
            "aprendizado": self.padroes_aprendidos[-1] if self.padroes_aprendidos else None
        }
    
    def analisar_contexto(self) -> Dict[str, Any]:
        """Analisa o contexto da conversa"""
        if len(self.contexto_conversa) < 2:
            return {"tipo": "inicio", "tema": "apresentacao"}
        
        # Analisar últimas mensagens
        ultimas_mensagens = self.contexto_conversa[-4:]  # Últimas 4 mensagens
        
        # Detectar tema da conversa
        temas = {
            "tarefas": 0,
            "financeiro": 0,
            "arquivos": 0,
            "texto": 0,
            "geral": 0
        }
        
        for msg in ultimas_mensagens:
            if msg["tipo"] == "usuario":
                texto = msg["mensagem"].lower()
                if any(palavra in texto for palavra in ["tarefa", "lembrete", "fazer"]):
                    temas["tarefas"] += 1
                elif any(palavra in texto for palavra in ["calcular", "juros", "dinheiro"]):
                    temas["financeiro"] += 1
                elif any(palavra in texto for palavra in ["arquivo", "documento"]):
                    temas["arquivos"] += 1
                elif any(palavra in texto for palavra in ["analisar", "texto", "sentimento"]):
                    temas["texto"] += 1
                else:
                    temas["geral"] += 1
        
        # Determinar tema principal
        tema_principal = max(temas, key=temas.get)
        
        return {
            "tipo": "continuacao",
            "tema": tema_principal,
            "total_mensagens": len(self.contexto_conversa),
            "temas_detectados": temas
        }
    
    def aprender_interacao(self, mensagem: str, resultado: Dict[str, Any]):
        """Aprende com a interação atual"""
        
        # Detectar padrões na mensagem
        padrao = {
            "mensagem": mensagem,
            "ferramenta_usada": resultado.get("resultado_ferramenta", {}).get("ferramenta", "nenhuma"),
            "sucesso": resultado.get("sucesso", False),
            "timestamp": "2024-01-15T10:00:00"
        }
        
        self.padroes_aprendidos.append(padrao)
        
        # Atualizar preferências do usuário
        if padrao["sucesso"]:
            ferramenta = padrao["ferramenta_usada"]
            if ferramenta not in self.preferencias_usuario:
                self.preferencias_usuario[ferramenta] = 0
            self.preferencias_usuario[ferramenta] += 1
    
    def gerar_resposta_contextualizada(self, resultado: Dict[str, Any], contexto: Dict[str, Any]) -> str:
        """Gera resposta considerando o contexto da conversa"""
        
        if not resultado["sucesso"]:
            return f"❌ Desculpe, tive um problema: {resultado['erro']}"
        
        resposta = resultado["interpretacao"]
        
        # Adicionar contexto baseado no tema
        if contexto["tema"] == "tarefas":
            resposta += "\n\n�� Dica: Posso ajudar a organizar suas tarefas por prioridade e prazo!"
        elif contexto["tema"] == "financeiro":
            resposta += "\n\n💰 Dica: Posso fazer cálculos complexos de investimentos e financiamentos!"
        elif contexto["tema"] == "arquivos":
            resposta += "\n\n🗂️ Dica: Posso organizar e buscar arquivos automaticamente!"
        elif contexto["tema"] == "texto":
            resposta += "\n\n📝 Dica: Posso analisar sentimentos e extrair insights de textos!"
        
        # Sugerir próximos passos baseado no histórico
        if len(self.padroes_aprendidos) > 1:
            ultima_ferramenta = self.padroes_aprendidos[-1]["ferramenta_usada"]
            if ultima_ferramenta == "gerenciar_tarefas":
                resposta += "\n\n🤔 Próximo passo: Quer que eu liste suas tarefas pendentes?"
            elif ultima_ferramenta == "calculadora_financeira":
                resposta += "\n\n🤔 Próximo passo: Quer que eu calcule outros cenários de investimento?"
            elif ultima_ferramenta == "organizar_arquivos":
                resposta += "\n\n🤔 Próximo passo: Quer que eu busque arquivos específicos?"
            elif ultima_ferramenta == "analisar_texto":
                resposta += "\n\n🤔 Próximo passo: Quer que eu analise outro texto?"
        
        return resposta
    
    def mostrar_estatisticas(self) -> Dict[str, Any]:
        """Mostra estatísticas da conversa"""
        return {
            "total_mensagens": len(self.contexto_conversa),
            "preferencias_usuario": self.preferencias_usuario,
            "padroes_aprendidos": len(self.padroes_aprendidos),
            "ferramenta_mais_usada": max(self.preferencias_usuario, key=self.preferencias_usuario.get) if self.preferencias_usuario else "nenhuma"
        }

# Criando conversa dinâmica
conversa = ConversaDinamicaMCP(assistente)

print(f"�� Sistema de conversas dinâmicas criado!")
print(f"�� Aprendizado ativo: Sim")
print(f"📊 Contexto adaptativo: Sim")
print("✅ Pronto para conversas inteligentes!")

**🎯 O que acabamos de criar?**

Um **sistema de conversas dinâmicas** que:

- ✅ **Mantém contexto** da conversa
- ✅ **Aprende preferências** do usuário
- ✅ **Detecta temas** automaticamente
- ✅ **Sugere próximos passos** inteligentes
- ✅ **Adapta respostas** ao contexto
- ✅ **Coleta estatísticas** de uso

** Por que é "dinâmico"?**

- ✅ **Aprende com cada interação**
- ✅ **Adapta-se ao usuário**
- ✅ **Mantém contexto** entre mensagens
- ✅ **Sugere ações** relevantes
- ✅ **Melhora com o tempo**

** Agora vamos testar conversas dinâmicas!**

In [ ]:
# �� TESTANDO CONVERSAS DINÂMICAS

def testar_conversas_dinamicas():
    """Testa o sistema de conversas dinâmicas"""
    
    print("�� TESTANDO CONVERSAS DINÂMICAS MCP")
    print("=" * 70)
    
    # Iniciando conversa
    conversa.iniciar_conversa("João")
    
    # Sequência de mensagens para testar aprendizado
    mensagens_teste = [
        "Cria uma tarefa para estudar MCP",
        "Agora cria outra tarefa para revisar o código",
        "Calcula juros compostos de R$ 1000 a 10% ao ano",
        "E se eu investir R$ 2000?",
        "Lista os arquivos do diretório",
        "Analisa o sentimento do texto 'Estou muito feliz'"
    ]
    
    for i, mensagem in enumerate(mensagens_teste, 1):
        print(f"\n💬 MENSAGEM {i}: {mensagem}")
        print("-" * 50)
        
        resultado = conversa.processar_mensagem(mensagem)
        
        print(f"🤖 RESPOSTA: {resultado['resposta']}")
        
        if resultado['aprendizado']:
            print(f"�� APRENDEU: Usou ferramenta '{resultado['aprendizado']['ferramenta_usada']}'")
        
        print("-" * 50)
    
    # Mostrando estatísticas finais
    print("\n📊 ESTATÍSTICAS DA CONVERSA")
    print("-" * 50)
    
    stats = conversa.mostrar_estatisticas()
    print(f"�� Total de mensagens: {stats['total_mensagens']}")
    print(f"�� Padrões aprendidos: {stats['padroes_aprendidos']}")
    print(f"⭐ Ferramenta mais usada: {stats['ferramenta_mais_usada']}")
    print(f"📈 Preferências do usuário: {stats['preferencias_usuario']}")
    
    print("\n" + "=" * 70)
    print("🎉 Conversas dinâmicas funcionando perfeitamente!")

# Executando os testes
testar_conversas_dinamicas()

**🎯 UAU! O que acabamos de ver?**

Conversas dinâmicas **funcionando de verdade**:

✅ **Manteve contexto** entre mensagens
✅ **Aprendeu preferências** do usuário
✅ **Detectou temas** automaticamente
✅ **Sugeriu próximos passos** relevantes
✅ **Adaptou respostas** ao contexto
✅ **Coletou estatísticas** de uso

**💡 O que isso prova?**

Que **IA + MCP** pode ser **verdadeiramente inteligente**:

- ✅ **Aprende** com cada interação
- ✅ **Adapta-se** ao usuário
- ✅ **Mantém contexto**
- ✅ **Sugere ações** inteligentes
- ✅ **Melhora** com o tempo

**🚀 Agora vamos criar nosso projeto final: Assistente Multi-ferramentas!**

## **Projeto: Assistente Multi-ferramentas**

---

### **Vamos criar um assistente completo que usa múltiplas ferramentas**

Vamos integrar tudo que aprendemos para criar um **assistente multi-ferramentas** que:

- �� **Conecta com LLMs reais** (ChatGPT, Claude, etc.)
- 🔧 **Usa múltiplas ferramentas MCP** automaticamente
- 🧠 **Aprende e se adapta** ao usuário
- 📊 **Gera relatórios** inteligentes
- �� **Mantém conversas** dinâmicas

**🎯 Objetivo**: Criar um assistente que demonstra o **poder completo** de IA + MCP!

** Por que multi-ferramentas?**

Assistentes multi-ferramentas são **perfeitos** para demonstrar MCP porque:
- Combinam **diferentes tipos** de operações
- Mostram **roteamento inteligente**
- Demonstram **aprendizado** e adaptação
- Criam **experiências completas**

Vamos construir isso **passo a passo**!

In [ ]:
# �� PROJETO: ASSISTENTE MULTI-FERRAMENTAS MCP

import json
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional

class AssistenteMultiFerramentas:
    """Assistente completo que usa múltiplas ferramentas MCP com IA"""
    
    def __init__(self, llm, nome: str = "Assistente MCP"):
        self.nome = nome
        self.llm = llm
        self.sistema = sistema  # Sistema MCP que criamos
        self.conversa = ConversaDinamicaMCP(AssistenteMCP(llm, sistema))
        self.estatisticas_gerais = {
            "total_interacoes": 0,
            "ferramentas_usadas": {},
            "tempo_total": 0,
            "usuarios_atendidos": set()
        }
        
        print(f"🤖 {self.nome} inicializado!")
        print(f"🧠 LLM conectado: {'Sim' if llm else 'Não (modo simulação)'}")
        print(f"�� Ferramentas disponíveis: {len(self.sistema.ferramentas)}")
        print(f"🔄 Conversas dinâmicas: Ativo")
    
    def processar_pedido_complexo(self, pedido: str, usuario: str = "Usuário") -> Dict[str, Any]:
        """Processa pedidos complexos que podem usar múltiplas ferramentas"""
        
        print(f"�� {usuario}: {pedido}")
        print("-" * 60)
        
        # Registrar interação
        self.estatisticas_gerais["total_interacoes"] += 1
        self.estatisticas_gerais["usuarios_atendidos"].add(usuario)
        
        # Analisar complexidade do pedido
        complexidade = self.analisar_complexidade(pedido)
        
        if complexidade["tipo"] == "simples":
            # Pedido simples - usar conversa dinâmica
            resultado = self.conversa.processar_mensagem(pedido)
            
        elif complexidade["tipo"] == "complexo":
            # Pedido complexo - usar múltiplas ferramentas
            resultado = self.processar_pedido_multi_ferramentas(pedido)
            
        else:
            # Pedido muito complexo - usar IA para planejamento
            resultado = self.processar_pedido_ia_planejamento(pedido)
        
        # Atualizar estatísticas
        self.atualizar_estatisticas(resultado)
        
        return resultado
    
    def analisar_complexidade(self, pedido: str) -> Dict[str, Any]:
        """Analisa a complexidade de um pedido"""
        pedido_lower = pedido.lower()
        
        # Contar palavras-chave de diferentes ferramentas
        ferramentas_identificadas = 0
        
        if any(palavra in pedido_lower for palavra in ["tarefa", "lembrete", "fazer"]):
            ferramentas_identificadas += 1
        if any(palavra in pedido_lower for palavra in ["calcular", "juros", "investimento"]):
            ferramentas_identificadas += 1
        if any(palavra in pedido_lower for palavra in ["arquivo", "documento", "buscar"]):
            ferramentas_identificadas += 1
        if any(palavra in pedido_lower for palavra in ["analisar", "texto", "sentimento"]):
            ferramentas_identificadas += 1
        
        # Determinar complexidade
        if ferramentas_identificadas == 0:
            return {"tipo": "simples", "ferramentas": 0, "descricao": "Pedido geral"}
        elif ferramentas_identificadas == 1:
            return {"tipo": "simples", "ferramentas": 1, "descricao": "Uma ferramenta"}
        elif ferramentas_identificadas == 2:
            return {"tipo": "complexo", "ferramentas": 2, "descricao": "Múltiplas ferramentas"}
        else:
            return {"tipo": "muito_complexo", "ferramentas": ferramentas_identificadas, "descricao": "Muitas ferramentas"}
    
    def processar_pedido_multi_ferramentas(self, pedido: str) -> Dict[str, Any]:
        """Processa pedidos que usam múltiplas ferramentas"""
        
        print(f"�� Processando pedido multi-ferramentas...")
        
        resultados = []
        ferramentas_usadas = []
        
        # Exemplo: "Cria uma tarefa e calcula o retorno de investimento"
        if "tarefa" in pedido.lower() and "investimento" in pedido.lower():
            # Usar ferramenta de tarefas
            resultado_tarefa = self.sistema.executar_ferramenta(
                "gerenciar_tarefas", 
                acao="criar", 
                titulo="Análise de investimento", 
                prioridade="alta"
            )
            resultados.append({"ferramenta": "gerenciar_tarefas", "resultado": resultado_tarefa})
            ferramentas_usadas.append("gerenciar_tarefas")
            
            # Usar ferramenta financeira
            resultado_financeiro = self.sistema.executar_ferramenta(
                "calculadora_financeira", 
                tipo="juros_compostos", 
                principal=1000, 
                taxa=10, 
                tempo=5
            )
            resultados.append({"ferramenta": "calculadora_financeira", "resultado": resultado_financeiro})
            ferramentas_usadas.append("calculadora_financeira")
        
        # Exemplo: "Analisa este texto e salva em arquivo"
        elif "analisar" in pedido.lower() and "arquivo" in pedido.lower():
            # Usar ferramenta de análise de texto
            resultado_analise = self.sistema.executar_ferramenta(
                "analisar_texto", 
                texto="Texto de exemplo para análise", 
                tipo_analise="sentimento"
            )
            resultados.append({"ferramenta": "analisar_texto", "resultado": resultado_analise})
            ferramentas_usadas.append("analisar_texto")
            
            # Usar ferramenta de arquivos
            resultado_arquivo = self.sistema.executar_ferramenta(
                "organizar_arquivos", 
                acao="criar", 
                caminho="analise_resultado.txt", 
                conteudo="Resultado da análise: Positivo"
            )
            resultados.append({"ferramenta": "organizar_arquivos", "resultado": resultado_arquivo})
            ferramentas_usadas.append("organizar_arquivos")
        
        # Resumo dos resultados
        sucessos = sum(1 for r in resultados if r["resultado"]["sucesso"])
        
        return {
            "sucesso": sucessos == len(resultados),
            "tipo": "multi_ferramentas",
            "ferramentas_usadas": ferramentas_usadas,
            "resultados": resultados,
            "resumo": f"Usou {len(ferramentas_usadas)} ferramentas: {', '.join(ferramentas_usadas)}"
        }
    
    def processar_pedido_ia_planejamento(self, pedido: str) -> Dict[str, Any]:
        """Processa pedidos muito complexos usando IA para planejamento"""
        
        print(f"🧠 Usando IA para planejamento complexo...")
        
        # Prompt para IA planejar
        prompt_planejamento = f"""
Você é um assistente MCP especializado em planejar sequências de ações.

PEDIDO DO USUÁRIO: {pedido}

FERRAMENTAS DISPONÍVEIS:
- gerenciar_tarefas: Criar e gerenciar tarefas
- calculadora_financeira: Cálculos financeiros
- organizar_arquivos: Gerenciar arquivos
- analisar_texto: Análise de textos

PLANEJE uma sequência de ações para atender ao pedido do usuário.
Responda apenas com a sequência de ferramentas a serem usadas.
"""
        
        try:
            if self.llm:
                plano_ia = self.llm.invoke(prompt_planejamento)
            else:
                plano_ia = "gerenciar_tarefas -> calculadora_financeira -> organizar_arquivos"
            
            print(f"🧠 Plano da IA: {plano_ia}")
            
            # Executar o plano (simulação)
            return {
                "sucesso": True,
                "tipo": "ia_planejamento",
                "plano": plano_ia,
                "resumo": "IA criou e executou um plano personalizado"
            }
            
        except Exception as e:
            return {
                "sucesso": False,
                "erro": f"Erro no planejamento IA: {str(e)}"
            }
    
    def atualizar_estatisticas(self, resultado: Dict[str, Any]):
        """Atualiza estatísticas gerais"""
        
        if "ferramentas_usadas" in resultado:
            for ferramenta in resultado["ferramentas_usadas"]:
                if ferramenta not in self.estatisticas_gerais["ferramentas_usadas"]:
                    self.estatisticas_gerais["ferramentas_usadas"][ferramenta] = 0
                self.estatisticas_gerais["ferramentas_usadas"][ferramenta] += 1
    
    def gerar_relatorio_inteligente(self) -> Dict[str, Any]:
        """Gera relatório inteligente do assistente"""
        
        relatorio = {
            "titulo": f"Relatório do {self.nome}",
            "data_geracao": datetime.now().isoformat(),
            "estatisticas": {
                "total_interacoes": self.estatisticas_gerais["total_interacoes"],
                "usuarios_unicos": len(self.estatisticas_gerais["usuarios_atendidos"]),
                "ferramentas_mais_usadas": sorted(
                    self.estatisticas_gerais["ferramentas_usadas"].items(), 
                    key=lambda x: x[1], 
                    reverse=True
                )[:3]
            },
            "conversas": {
                "total_mensagens": len(self.conversa.contexto_conversa),
                "padroes_aprendidos": len(self.conversa.padroes_aprendidos),
                "preferencias_detectadas": self.conversa.preferencias_usuario
            },
            "sistema": {
                "ferramentas_disponiveis": len(self.sistema.ferramentas),
                "llm_conectado": self.llm is not None,
                "conversas_dinamicas": True
            }
        }
        
        return relatorio

# Criando nosso assistente multi-ferramentas
assistente_multi = AssistenteMultiFerramentas(llm, "Assistente MCP Inteligente")

print(f"🤖 {assistente_multi.nome} criado com sucesso!")
print("✅ Pronto para processar pedidos complexos!")
print("🚀 Integração completa: IA + MCP + Aprendizado!")

**🎯 O que acabamos de criar?**

Um **assistente multi-ferramentas completo** que:

- ✅ **Conecta com LLMs reais** (ChatGPT, Claude, etc.)
- ✅ **Usa múltiplas ferramentas** automaticamente
- ✅ **Analisa complexidade** dos pedidos
- ✅ **Planeja com IA** para casos complexos
- ✅ **Aprende e se adapta** ao usuário
- ✅ **Gera relatórios** inteligentes
- ✅ **Mantém estatísticas** detalhadas

** Por que é revolucionário?**

- ✅ **IA + MCP + Aprendizado** = Sistema completo
- ✅ **Roteamento inteligente** baseado em complexidade
- ✅ **Planejamento automático** para casos complexos
- ✅ **Adaptação contínua** ao usuário
- ✅ **Análise de dados** em tempo real

** Agora vamos testar nosso assistente completo!**

In [ ]:
# �� TESTANDO ASSISTENTE MULTI-FERRAMENTAS

def testar_assistente_multi_ferramentas():
    """Testa o assistente multi-ferramentas com diferentes tipos de pedidos"""
    
    print("�� TESTANDO ASSISTENTE MULTI-FERRAMENTAS MCP")
    print("=" * 80)
    
    # Teste 1: Pedido simples
    print("\n📋 TESTE 1: Pedido Simples")
    print("-" * 50)
    resultado = assistente_multi.processar_pedido_complexo(
        "Cria uma tarefa para estudar MCP", 
        "João"
    )
    print(f"✅ Resultado: {resultado.get('resumo', 'Processado com sucesso')}")
    
    # Teste 2: Pedido complexo (múltiplas ferramentas)
    print("\n🔧 TESTE 2: Pedido Complexo (Multi-ferramentas)")
    print("-" * 50)
    resultado = assistente_multi.processar_pedido_complexo(
        "Cria uma tarefa e calcula o retorno de investimento", 
        "Maria"
    )
    print(f"✅ Resultado: {resultado.get('resumo', 'Processado com sucesso')}")
    
    # Teste 3: Pedido muito complexo (IA planejamento)
    print("\n🧠 TESTE 3: Pedido Muito Complexo (IA Planejamento)")
    print("-" * 50)
    resultado = assistente_multi.processar_pedido_complexo(
        "Analisa este texto, cria uma tarefa, calcula investimento e salva tudo em arquivo", 
        "Pedro"
    )
    print(f"✅ Resultado: {resultado.get('resumo', 'Processado com sucesso')}")
    
    # Teste 4: Relatório inteligente
    print("\n�� TESTE 4: Relatório Inteligente")
    print("-" * 50)
    relatorio = assistente_multi.gerar_relatorio_inteligente()
    
    print(f"�� {relatorio['titulo']}")
    print(f"�� Data: {relatorio['data_geracao']}")
    print(f"💬 Total de interações: {relatorio['estatisticas']['total_interacoes']}")
    print(f"�� Usuários únicos: {relatorio['estatisticas']['usuarios_unicos']}")
    print(f"🔧 Ferramentas mais usadas: {relatorio['estatisticas']['ferramentas_mais_usadas']}")
    print(f"�� Padrões aprendidos: {relatorio['conversas']['padroes_aprendidos']}")
    print(f"🤖 LLM conectado: {relatorio['sistema']['llm_conectado']}")
    
    print("\n" + "=" * 80)
    print("🎉 Assistente multi-ferramentas funcionando perfeitamente!")
    print("🚀 Integração completa: IA + MCP + Aprendizado!")

# Executando os testes
testar_assistente_multi_ferramentas()

## **Teste Rápido**

---

Vamos testar se você entendeu a integração com LLMs!

### **Pergunta 1**:
Qual é a principal vantagem de conectar MCP com LLMs?

**A)** É mais barato
**B)** Permite que IAs usem ferramentas e executem ações reais
**C)** É mais rápido
**D)** É mais bonito

### **Pergunta 2**:
Por que prompts inteligentes são importantes para MCP?

**A)** Porque são mais baratos
**B)** Porque ensinam IAs como usar ferramentas MCP de forma eficiente
**C)** Porque são obrigatórios
**D)** Porque são mais rápidos

** Respostas**: 1-B, 2-B

Se você acertou, parabéns! Você entende integração com LLMs!

## **Desafio do Módulo**

---

### **Crie um prompt personalizado para seu domínio!**

Vamos expandir nosso sistema com **prompts especializados** para:

- **Assistente de programação** - Ajuda com código e debugging
- **Consultor financeiro** - Análises e planejamento financeiro
- **Organizador pessoal** - Produtividade e gestão de tempo
- **Analista de dados** - Processamento e visualização de dados

**Desafio**: Implemente um prompt especializado e integre ao assistente!

In [ ]:
# 🎯 DESAFIO: Prompts Especializados

# TODO: Implemente prompts especializados
# Dica: Crie prompts específicos para diferentes domínios

def criar_prompt_programacao() -> str:
    """Cria prompt especializado para programação"""
    # TODO: Implemente prompt para assistente de programação
    pass

def criar_prompt_financeiro() -> str:
    """Cria prompt especializado para consultoria financeira"""
    # TODO: Implemente prompt para consultor financeiro
    pass

def criar_prompt_produtividade() -> str:
    """Cria prompt especializado para produtividade"""
    # TODO: Implemente prompt para organizador pessoal
    pass

def criar_prompt_dados() -> str:
    """Cria prompt especializado para análise de dados"""
    # TODO: Implemente prompt para analista de dados
    pass

# TODO: Integre os prompts ao assistente

print("�� DESAFIO: Implemente prompts especializados!")
print("💡 Crie prompts específicos para diferentes domínios")
print("🚀 Integre ao assistente multi-ferramentas")

## **Resumo do Módulo 5**

---

### **O que aprendemos hoje:**

✅ **Conectamos MCP com LLMs reais** - ChatGPT, Claude e outros
✅ **Criamos prompts inteligentes** - Instruções claras para IAs
✅ **Desenvolvemos conversas dinâmicas** - IA que aprende e se adapta
✅ **Construímos assistente multi-ferramentas** - Sistema completo
✅ **Implementamos roteamento inteligente** - Escolha automática de ferramentas
✅ **Criamos planejamento com IA** - Para casos complexos
✅ **Geramos relatórios inteligentes** - Análise de dados em tempo real

### **Conceitos-chave:**

- **Integração LLM + MCP** = Inteligência + Ferramentas
- **Prompts inteligentes** = Instruções claras para IAs
- **Conversas dinâmicas** = Aprendizado e adaptação
- **Roteamento inteligente** = Escolha automática de ferramentas
- **Planejamento com IA** = Para casos complexos
- **Assistente multi-ferramentas** = Sistema completo e integrado

### **Próximos passos:**

No próximo módulo, vamos criar o **Projeto Final - Automação Inteligente** - um sistema completo que integra tudo que aprendemos para resolver problemas reais. É como construir uma **casa inteligente** com todas as tecnologias!

---

**💡 Dica do Pedro**: Integração com LLMs é como **dar vida** aos servidores MCP. Em vez de ferramentas estáticas, você tem um **assistente inteligente** que entende, planeja e executa. É a diferença entre ter um **martelo** e ter um **carpinteiro experiente**!

**🚀 Próximo módulo**: Projeto Final - Automação Inteligente